In [1]:
import csv
import numpy as np
import pandas as pd

import cv2

import keras
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [2]:
# remove prefixes
import os
path = '/Users/take5v/Repository/data/data4stas/01_data_cls/'
os.chdir(path)

In [ ]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

In [3]:
def process_file(path):
    df = pd.read_csv(path)
    
    x = []
    y = []

    for index, row in df.iterrows():
        x.append(cv2.imread(row['FilePath'], cv2.IMREAD_GRAYSCALE))
        y.append(float(row['ClassId']))

    x = np.array(x)
    y = np.array(y)
    
    num_classes = len(np.unique(y))

    x = x.reshape(x.shape + (-1,))
    y = y - 1
    y = keras.utils.to_categorical(y, num_classes=num_classes)

    return x, y, num_classes
            

def process_line(line):
    pass

def generate_arrays_from_file(path):
    df = pd.read_csv(path)
    
    while 1:
        f = open(path)
        for line in f:
            # create numpy arrays of input data
            # and labels, from each line in the file
            x1, x2, y = process_line(line)
            yield ({'input_1': x1, 'input_2': x2}, {'output': y})
        f.close()

In [4]:
x, y, num_classes = process_file('Alexander_cls_train.csv')

In [5]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
    fill_mode='reflect')

In [6]:
datagen.fit(x)

/usr/local/lib/python2.7/site-packages/keras/preprocessing/image.py:653: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (555, 200, 200, 1) (1 channels).
  ' (' + str(x.shape[self.channel_axis]) + ' channels).')


In [ ]:
%pylab inline

In [26]:
model = Sequential()
# input: 200x200 grayscale images -> (200, 200, 1) tensors.
# this applies 8 convolution filters of size 3x3 each.
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(200, 200, 1)))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

# model.add(Conv2D(64, (2, 2), kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(128, (2, 2), kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(256, (2, 2), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Flatten())
# model.add(Dense(2048, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model.add(Dense(num_classes, kernel_regularizer=regularizers.l2(0.01), activation='softmax'), )
model.add(Dropout(0.5))

# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# model.fit(x, y, batch_size=8, epochs=10, validation_data=(x_train, y_train))
model.fit(x, y, batch_size=32, epochs=10)

In [28]:
model.fit_generator(datagen.flow(x, y, batch_size=32),
                    steps_per_epoch=len(x) / 32, epochs=300)

Epoch 1/300
17/17 [==============================] - 14s - loss: 12.8359 - acc: 0.0037    
Epoch 2/300
17/17 [==============================] - 12s - loss: 11.7696 - acc: 0.0037    
Epoch 3/300
17/17 [==============================] - 12s - loss: 11.2831 - acc: 0.0074    
Epoch 4/300
17/17 [==============================] - 13s - loss: 10.9030 - acc: 0.0055    
Epoch 5/300
17/17 [==============================] - 12s - loss: 10.6714 - acc: 0.0147    
Epoch 6/300
17/17 [==============================] - 12s - loss: 9.9886 - acc: 0.0037    
Epoch 7/300
17/17 [==============================] - 14s - loss: 10.1983 - acc: 0.0092    
Epoch 8/300
17/17 [==============================] - 13s - loss: 9.9939 - acc: 0.0184     
Epoch 9/300
17/17 [==============================] - 12s - loss: 10.4264 - acc: 0.0107    
Epoch 10/300
17/17 [==============================] - 12s - loss: 9.9599 - acc: 0.0184    
Epoch 11/300
17/17 [==============================] - 12s - loss: 9.7679 - acc: 0.0111    


In [30]:
test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True
)

test_paths = ['Alexander_cls_train.csv',
             'Alexander_cls_test.csv',
             'Alexander_cls_test1.csv',
             'Alexander_cls_test2.csv',
             'Alexander_cls_test3.csv',
             'Alexander_cls_test4.csv']

for test_path in test_paths:
    x_test, y_test, num_classes = process_file(test_path)
    test_datagen.fit(x_test)
    score = model.evaluate_generator(test_datagen.flow(x_test, y_test, batch_size=32), steps=len(x_test) / 32)
    print test_path + ' : ' + ', '.join(str(x) for x in score)

Alexander_cls_train.csv : 1.57316931556, 0.726102941176
Alexander_cls_test.csv : 4.03173589706, 0.321557971014
Alexander_cls_test1.csv : 2.00173806443, 0.5625
Alexander_cls_test2.csv : 7.46967282015, 0.0496323529412
Alexander_cls_test3.csv : 2.92800291847, 0.330882352941
Alexander_cls_test4.csv : 3.76812979754, 0.332720588235


In [ ]:
model.evaluate(x, y, batch_size=32)

In [ ]:
y_predict = model.predict(x_train, batch_size=32)

In [ ]:
np.unique(np.argmax(y_predict, axis=1))

In [31]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_48 (Conv2D)           (None, 198, 198, 32)      320       
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 66, 66, 32)        0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 21, 21, 64)        0         
_________________________________________________________________
dropout_37 (Dropout)         (None, 21, 21, 64)        0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 19, 19, 128)       73856     
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 6, 6, 128)         0         
__________

In [ ]:
# спросить про 500 samples на класс (хорошо, но можно оверфитнуться)
# когда 50 сэмплов хорошо, обычно плохо